In [ ]:
import pandas as pd
import os
import re
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', 50)

df = pd.DataFrame()

folder = "groupbuys"
# folder2 = "sorted-preprocessed-data"
folder2 = "raw-data"
#load every csv in the folder and append them
csv = os.listdir(folder2+"/"+folder)
for file in csv:
    df = pd.concat([df, pd.read_csv(folder2+"/"+folder+"/"+file)], axis=0)

# df = df[["topic","post","start_edit", "last_edit"]]
df["post"] = df["post"].astype(str)
df["original_info"] = df["original_info"].astype(str)
# df = df.sort_values(by=["start_edit"]).reset_index(drop=True)
df = df.reset_index(drop=True)

def clean_post(post):
    #remove all links
    post = re.sub(r'http\S+', '<link>', post)
    #remove duplicate spaces
    post = re.sub(' +', ' ', post)
    #remove all non-ascii characters
    post = post.encode("ascii", errors="ignore").decode()
    #remove all words that have a length above 20 (most likely a btc address or something useless)
    post = " ".join([(word if len(word) < 20 else "<btc_address>") for word in post.split()])

    return post

df["post"] = df["post"].apply(clean_post)

In [ ]:
df

In [ ]:
df_ = df[df["topic"] == "Re: Avalon resell and hosting plan"]
for (id,row) in df_.iterrows():
    html = row["original_info"]
    soup = BeautifulSoup(html, 'html.parser')
    #the date is in <div class="smalltext">
    date = soup.find("div", {"class": "smalltext"}).text
    #delete <div class="quote">
    for quote in soup.find_all("div", {"class": "quote"}):
        quote.decompose()
    for quote in soup.find_all("div", {"class": "quoteheader"}):
        quote.decompose()
    #post is in <div class="post">
    post = soup.find("div", {"class": "post"}).text
    print("date:", date)
    print(clean_post(post))
    print("----")

In [ ]:
# group by topic and append all the strings from the post column
# df = df.groupby('topic').agg({'post': '\n--------\n'.join, 'start_edit': 'first', 'last_edit': 'last'}).reset_index()
df = df.groupby('topic').agg({'post': '<sep>'.join, 'start_edit': '<sep>'.join, 'last_edit': '<sep>'.join}).reset_index()

In [ ]:
df = df.sort_values(by=["start_edit"])

In [ ]:
df

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(df["start_edit"], bins=10, edgecolor='black')

# # Add labels and title
# plt.xlabel('date')
# plt.ylabel('Value')
# plt.title('Bar Plot of Integers')

# # Show the plot
# plt.show()

# print(df["start_edit"].max())
# print(df["start_edit"].min())

In [ ]:
#print a sample
for (id,row) in df.head(50).iterrows():
    if(len(row["post"]) < 100):
        continue
    print("id: ", id)
    print("topic: ", row["topic"])
    print("posts:")
    # print("date: ", row["start_edit"])
    # print(row["post"])
    # print("\n\n\n\n\n")
    for (post, start_edit, last_edit) in zip(row["post"].split("<sep>"), row["start_edit"].split("<sep>"), row["last_edit"].split("<sep>")):
        print("start_edit: ", start_edit)
        print("last_edit: ", last_edit)
        print(post)
        print("-----------------")
    print("\n\n\n\n\n")

In [ ]:
print(df.loc[2072]["post"])

In [ ]:
print(df.loc[3529]["post"])
print(df.loc[2072]["post"])